In [1]:
# lets import some stuff..
from meshparty import trimesh_io, trimesh_vtk
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import vtk

In [2]:
# setup the mesh meta to handle downloads and caching
mesh_dir = 'data/meshes'
seg_source = "precomputed://https://storage.googleapis.com/microns_public_datasets/pinky100_v185/seg"
mm = trimesh_io.MeshMeta(cv_path=seg_source,
                         disk_cache_path='mesh_dir', 
                         cache_size=20)

syn_df = pd.read_csv('data/soma_subgraph_synapses_spines_v185.csv')

In [3]:
syn_df.head()

,id,pre_root_id,post_root_id,cleft_vx,spine_vol_um3,ctr_pt_x_nm,ctr_pt_y_nm,ctr_pt_z_nm,pre_pos_x_vx,pre_pos_y_vx,pre_pos_z_vx,ctr_pos_x_vx,ctr_pos_y_vx,ctr_pos_z_vx,post_pos_x_vx,post_pos_y_vx,post_pos_z_vx
0,1484,648518346349539437,648518346349531254,798,0.133004,365476,231192,63280,91332,57836,1584,91369,57798,1582,91332,57748,1584
1,3056393,648518346349537978,648518346349531254,209,0.087794,312120,209816,9960,78050,52470,248,78030,52454,249,77980,52444,249
2,310116,648518346349538410,648518346349531254,869,0.234537,313596,185764,23920,78314,46428,598,78399,46441,598,78404,46410,598
3,1533059,648518346349538410,648518346349531254,231,0.081921,289560,174904,33920,72340,43756,849,72390,43726,848,72354,43704,849
4,2505779,648518346349538410,648518346349531254,669,0.105706,296056,162304,9520,73966,40608,231,74014,40576,238,73918,40562,232


In [4]:
# download a cell mesh
seg_id = 648518346349531254

mesh = mm.mesh(seg_id=seg_id, remove_duplicate_vertices=True)

Existence Testing:   0%|          | 0/20 [00:00<?, ?it/s]
Downloading: 1it [00:00,  6.26it/s]
Downloading: 510it [00:05, 87.92it/s]                         
Decoding Mesh Buffer: 100%|██████████| 256/256 [00:00<00:00, 3781.66it/s]


In [27]:
mm.mesh?

In [7]:
# make an actor
mesh_actor = trimesh_vtk.mesh_actor(mesh)

In [9]:
# lets say we want to aim a camera at a particular synapse on this cell
syn_pos=syn_df.query(f'post_root_id=={seg_id}').iloc[0][['ctr_pt_x_nm', 'ctr_pt_y_nm', 'ctr_pt_z_nm']]
camera = trimesh_vtk.oriented_camera(syn_pos, backoff=10)
trimesh_vtk.render_actors([mesh_actor], camera=camera)

# now note when i adjust the camera dynamically
# my camera object is updated, so when i hit 'q'
# or close window, the camera has new parameters

(vtkRenderingOpenGL2Python.vtkOpenGLRenderer)0x1a2ee36440

In [10]:
# so when i run the camera again.. i get the same perspective
trimesh_vtk.render_actors([mesh_actor], camera=camera)

(vtkRenderingOpenGL2Python.vtkOpenGLRenderer)0x1a2ee36130

In [11]:
ren, cameras = trimesh_vtk.render_actors([mesh_actor], return_keyframes=True)

In [14]:
# cameras are by default perspectivetransformation
# but for figures you want to do orthographic, so scale is more meaningful
# across depths.
# to do this, you want to adjust the camera's mode to ParallelProjection
camera.SetParallelProjection(True)
# you have to set the scale of the camera
# which is half the height of the window
# in world units (so nm for our meshes)
camera.SetParallelScale(1500)
trimesh_vtk.render_actors([mesh_actor], camera=camera)

(vtkRenderingOpenGL2Python.vtkOpenGLRenderer)0x1a414c4980

In [15]:


# this is the built in LegendScaleActor.. it will dynamically update
# but has an ugly font and line and you can't control its size precisely
# reccomended workflow... take two renderings, with and without and use
# image with scale bar to create your own custom one in illustrator

legendScaleActor = vtk.vtkLegendScaleActor()
legendScaleActor.BottomAxisVisibilityOff()
legendScaleActor.TopAxisVisibilityOff()
legendScaleActor.LeftAxisVisibilityOff()
legendScaleActor.RightAxisVisibilityOff()
legendScaleActor.GetLegendTitleProperty().SetFontSize(50)
legendScaleActor.GetLegendTitleProperty().SetColor(0,0,0)


trimesh_vtk.render_actors([mesh_actor, legendScaleActor], camera=camera)


(vtkRenderingOpenGL2Python.vtkOpenGLRenderer)0x1a41a8e360

In [16]:
# save images, one with legendActor one without
# i also make the background gray in order to see the end of the scalebar better
trimesh_vtk.render_actors([mesh_actor, legendScaleActor], filename='image_with_scale.png',
                          back_color=(.5, .5, .5), do_save=True, camera=camera)
trimesh_vtk.render_actors([mesh_actor], filename='image_without.png', do_save=True, camera=camera)


(vtkRenderingOpenGL2Python.vtkOpenGLRenderer)0x1a41a8e2f0

In [ ]:
# this approach requires that you prespecify your window height in nm and pixels, and your scale bar size
# in nm. I'm not sure whether this is good enough to really merge into MeshParty, but it works here.

In [17]:
# vtk function to make a simple line actor in the lower left that is a certain number of pixels wide
def make_scalebar_actor(width_frac, line_width=5.0, color=(0,0,0), offset=.05):
    points = vtk.vtkPoints();
    points.SetNumberOfPoints(2);
    points.Allocate(2);

    points.InsertPoint(0, offset, offset, 0);
    points.InsertPoint(1, offset+width_frac, offset, 0);

    cells = vtk.vtkCellArray();
    cells.Initialize();

    line = vtk.vtkLine();
    line.GetPointIds().SetId(0, 0);
    line.GetPointIds().SetId(1, 1);

    cells.InsertNextCell(line);
    poly = vtk.vtkPolyData();
    poly.Initialize();
    poly.SetPoints(points);
    poly.SetLines(cells);
    poly.Modified();

    coordinate = vtk.vtkCoordinate();
    coordinate.SetCoordinateSystemToNormalizedDisplay();
    mapper = vtk.vtkPolyDataMapper2D();
    mapper.SetInputData(poly);
    mapper.SetTransformCoordinate(coordinate);
    mapper.ScalarVisibilityOn();
    mapper.SetScalarModeToUsePointData();
    mapper.Update();
    
    actor = vtk.vtkActor2D();
    actor.SetMapper(mapper);
    actor.GetProperty().SetLineWidth(line_width); 
    actor.GetProperty().SetColor(*color)
    
    return actor
# used for my testing
# scalebarActor = make_scalebar_actor(100)

In [18]:
# functions to make a camera and scale bar that are calibrated together

def make_scale_bar_from_camera(camera, scale_bar_nm, video_height=720, video_width=1440):
    
    scale_bar_frac_h = scale_bar_nm/camera.GetParallelScale()
    scale_bar_frac = scale_bar_frac_h * (video_height*1.0/(video_width*2))
    scale_bar_actor = make_scalebar_actor(scale_bar_frac)
    
    return  scale_bar_actor  

def make_orthographic_camera_and_scalebar(ctr_pt, window_height_nm, scale_bar_nm,
                                          video_width=1440, video_height=720):
    
    camera = trimesh_vtk.oriented_camera(ctr_pt, backoff_vector=1000)
    camera.SetParallelProjection(True)
    camera.SetParallelScale(window_height_nm)
    scale_bar_actor = make_scale_bar_from_camera(camera, scale_bar_nm,
                                                 video_height=video_height,
                                                 video_width=video_width)
    return camera, scale_bar_actor
 

In [20]:
# use these functions to make a camera scale bar pair
camera, scale_bar_actor = make_orthographic_camera_and_scalebar(syn_pos,
                                                                3000, 3000,
                                                                video_height=1440,
                                                                video_width=1080*2)

# these are other scale bar options
# this one places a scalebar in a specific 3d location
# sba = trimesh_vtk.scale_bar_actor(syn_pos, camera, length=3000)
# this is a second scalebar actor that helped convince me that orthographic view was working right
# sba2 = trimesh_vtk.scale_bar_actor(syn_pos+[0,10000,0], camera, length=3000)

# this is a 3d line that has end points in specific 3d locations, which convinced me the scalebar was right
# linea=trimesh_vtk.linked_point_actor(np.array([syn_pos]), np.array([syn_pos+[3000,0,0]]), line_width=5)

# here is a prettier way that uses the above function to make a 50um scale bar
# however, if you zoom at all the scalebar won't change and will not be 50um anymore
trimesh_vtk.render_actors([mesh_actor, scale_bar_actor],
                          camera=camera, video_height=1440, video_width=1080*2)

(vtkRenderingOpenGL2Python.vtkOpenGLRenderer)0x1a41a8ee50

In [22]:
# a brief tour through other visualization options
import seaborn as sns


# point cloud actor, for drawing lots of spheres

# down list to only synapses onto this cell
cell_syn_df = syn_df.query(f'post_root_id=={seg_id}')
# make an xyz array
syn_xyz=cell_syn_df[['ctr_pt_x_nm', 'ctr_pt_y_nm', 'ctr_pt_z_nm']].values
# make size 50*sqrt(cleft_size)
syn_size = 50*np.sqrt(cell_syn_df.cleft_vx.values)
# run sizes through a colormap
cmap = np.array(sns.color_palette('viridis', 1000))
# according to the log of the cleft size
# note you can use this function to map 'color' 
# onto meshes, skeleton nodes, etc...
# for explicit color passing, vtk needs Nx3 uint8 arrays from [0,255]
syn_color = trimesh_vtk.values_to_colors(np.log(cell_syn_df.cleft_vx.values),
                                         cmap)
# use point cloud actor function to make an actor
syn_actor = trimesh_vtk.point_cloud_actor(syn_xyz,
                                          size=syn_size,
                                          color=syn_color)




In [23]:
# visualize points and cell
trimesh_vtk.render_actors([syn_actor, mesh_actor])

(vtkRenderingOpenGL2Python.vtkOpenGLRenderer)0x1a38bc1bb0

In [24]:
# linked point actor, for drawing lots of lines connecting points

# make pre and post xyz arrays
pre_xyz=cell_syn_df[['pre_pos_x_vx', 'pre_pos_y_vx', 'pre_pos_z_vx']].values * np.array([4,4,40])
post_xyz =cell_syn_df[['post_pos_x_vx', 'post_pos_y_vx', 'post_pos_z_vx']].values * np.array([4,4,40])

syn_line_actor = trimesh_vtk.linked_point_actor(pre_xyz, post_xyz, line_width=5, opacity=1.0)


In [25]:
# lets aim the camera at first synapse to see it better.. backoff 4 microns
camera = trimesh_vtk.oriented_camera(syn_xyz[0,:], backoff=4)
trimesh_vtk.render_actors([syn_line_actor, mesh_actor], camera=camera)

(vtkRenderingOpenGL2Python.vtkOpenGLRenderer)0x1a41ac04b0